In [1]:
import numpy as np
import itertools
from itertools import product
import pandas as pd

In [2]:
def frecuencia_modos(nx, ny, nz, lx, ly, lz, c0=343):
    return (c0 / 2) * np.sqrt((nx / lx) ** 2 + (ny / ly) ** 2 + (nz / lz) ** 2)

def clasificar_modo(nx, ny, nz):
    num_ceros = sum([nx == 0, ny == 0, nz == 0])
    if num_ceros == 2:
        return "Axial"
    elif num_ceros == 1:
        return "Tangencial"
    else:
        return "Oblicuo"


def modos_degenerados(frecuencias, tolerancia=1.0):
    degenerados = []
    n = len(frecuencias)
    usados = set()
    for i in range(n):
        for j in range(i+1, n):
            if abs(frecuencias[i][3] - frecuencias[j][3]) <= tolerancia:
                for k in range(j+1, n):
                    if abs(frecuencias[i][3] - frecuencias[j][3]) <= tolerancia and abs(frecuencias[j][3] - frecuencias[k][3]) <= tolerancia:
                        degenerados.append((frecuencias[i], frecuencias[j], frecuencias[k]))
                        usados.update([i, j, k])
                        break
                else:
                    if i not in usados and j not in usados:
                        degenerados.append((frecuencias[i], frecuencias[j]))
                        usados.update([i, j])
    return degenerados

In [3]:
# Dimensiones del recinto en metros
lx, ly, lz = 4.11, 5.56, 2.49

In [ ]:
frecuencias = []
N_max = 20
f_max = 300  # Frecuencia máxima deseada (Hz)

for nx in range(N_max + 1):
    for ny in range(N_max + 1):
        for nz in range(N_max + 1):
            if (nx, ny, nz) == (0, 0, 0):
                continue  # Ignora el modo trivial
            f_m = frecuencia_modos(nx, ny, nz, lx, ly, lz)
            if f_m <= f_max:
                frecuencias.append((nx, ny, nz, f_m))

In [ ]:
# Clasificar y guardar en listas
nx_list, ny_list, nz_list, f_list, tipo_list = [], [], [], [], []

for nx, ny, nz, f in frecuencias:
    tipo = clasificar_modo(nx, ny, nz)
    nx_list.append(nx)
    ny_list.append(ny)
    nz_list.append(nz)
    f_list.append(f)
    tipo_list.append(tipo.lower())  # convertir a minúsculas para compatibilidad

# Crear DataFrame
modos_df = pd.DataFrame({
    'nx': nx_list,
    'ny': ny_list,
    'nz': nz_list,
    'frecuencia': f_list,
    'tipo': tipo_list
})

# Guardar como CSV
modos_df.to_csv("modos_sala.csv", index=False)
print("Archivo 'modos_sala.csv' exportado correctamente.")

Archivo 'modos_sala.csv' exportado correctamente.


In [6]:
# Identificar modos degenerados entre las primeras 30 frecuencias
tolerancia = 1.0 # Tolerancia en Hz
degenerados = modos_degenerados(frecuencias[:30], tolerancia)
print("\nModos degenerados dentro de la tolerancia de 1.0 Hz:")
for modo in degenerados:
    if len(modo) == 2:
        (nx1, ny1, nz1, f1), (nx2, ny2, nz2, f2) = modo
        print(f"Modo ({nx1},{ny1},{nz1}) - {f1:.2f} Hz y Modo ({nx2},{ny2},{nz2}) - {f2:.2f} Hz")
    elif len(modo) == 3:
        (nx1, ny1, nz1, f1), (nx2, ny2, nz2, f2), (nx3, ny3, nz3, f3) = modo
        print(f"Modo ({nx1},{ny1},{nz1}) - {f1:.2f} Hz , Modo ({nx2},{ny2},{nz2}) - {f2:.2f} Hz y Modo ({nx3},{ny3},{nz3}) - {f3:.2f} Hz")


Modos degenerados dentro de la tolerancia de 1.0 Hz:
Modo (0,0,3) - 206.63 Hz y Modo (0,5,2) - 206.79 Hz
Modo (0,1,2) - 141.16 Hz y Modo (0,4,1) - 141.30 Hz
Modo (0,2,1) - 92.46 Hz y Modo (0,3,0) - 92.54 Hz
Modo (0,4,2) - 184.93 Hz y Modo (0,6,0) - 185.07 Hz
